In [1]:
!pip install pandas
!pip install sqlalchemy
import sqlalchemy as sa


In [2]:
import sqlite3
import pandas as pd

# Create a new SQLite database
conn = sqlite3.connect('crime_data.db')
cur = conn.cursor()

# Create table schema - modify this according to the structure of your CSV
cur.execute('''
CREATE TABLE Crimes (
    id INTEGER PRIMARY KEY,
    case_number TEXT,
    date TEXT,
    block TEXT,
    iucr TEXT,
    primary_type TEXT,
    description TEXT,
    location_description TEXT,
    arrest BOOLEAN,
    domestic BOOLEAN,
    beat INTEGER,
    district INTEGER,
    ward INTEGER,
    community_area INTEGER,
    fbi_code TEXT,
    latitude REAL,
    longitude REAL
)
''')
conn.commit()

In [3]:
from google.colab import drive
drive.mount('/content/drive',)


Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/capstone/Crimes_2020.csv')

In [8]:

df.to_sql('Crimes', conn, if_exists='replace', index=False)

212249

In [12]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

app = Flask(__name__)
# Configure your database URI
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////path_to_your_db/crime_data.db'
db = SQLAlchemy(app)

class Crime(db.Model):
    __tablename__ = 'Crimes'
    id = db.Column(db.Integer, primary_key=True)
    case_number = db.Column(db.String(255))
    date = db.Column(db.DateTime)
    block = db.Column(db.String(255))
    iucr = db.Column(db.String(50))
    primary_type = db.Column(db.String(100))
    description = db.Column(db.String(255))
    location_description = db.Column(db.String(255))
    arrest = db.Column(db.Boolean)
    domestic = db.Column(db.Boolean)
    beat = db.Column(db.Integer)
    district = db.Column(db.Integer)
    ward = db.Column(db.Integer)
    community_area = db.Column(db.Integer)
    fbi_code = db.Column(db.String(50))
    latitude = db.Column(db.Float)
    longitude = db.Column(db.Float)
    # Add other fields as necessary

@app.route('/')
def index():
    # Example query: Get all crimes in 2020
    crimes_2020 = Crime.query.filter(db.func.strftime('%Y', Crime.date) == '2020').all()
    return render_template('main_page.html', crimes_2020=crimes_2020)

if __name__ == '__main__':
    app.run(debug=True)

TypeError: 'module' object is not callable